# DistilBART

**DistilBART** is a state-of-the-art language model that has been trained specifically for text summarization tasks. Here are some pros and cons of using DistilBART for text summarization of news articles:

### Pros:

* DistilBART can generate high-quality summaries of news articles that capture the most important information in the text.

* It can save time and effort for human summarizers by automatically producing summaries of news articles in a matter of seconds.

* DistilBART can summarize news articles in multiple languages, making it a versatile tool for news organizations that operate in multilingual markets.

* It can scale to handle large volumes of news articles, making it suitable for use in automated news aggregation systems.

### Cons:

* DistilBART is trained on a large corpus of text, which means that it may not always capture the nuances of specific news articles.

* Like all machine learning models, DistilBART can be prone to bias, which may result in summaries that reflect the biases of the training data.

* DistilBART can sometimes produce summaries that are overly simplistic or miss important details, which may lead to a loss of context.

* It can struggle with certain types of news articles, such as those that contain a lot of technical or scientific jargon.

Overall, DistilBART is a powerful tool for text summarization of news articles, but it is important to use it in conjunction with human oversight to ensure that the summaries are accurate and reflect the full context of the original articles.

These are the scores we achieved:

    ROUGE Score:
    Precision: 1.000
    Recall: 0.358
    F1-Score: 0.527

    BLEU Score: 0.825

## References

Here are some papers related to DistilBART:

1. "DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter" by Victor Sanh, Lysandre Debut, Julien Chaumond, and Thomas Wolf. This paper introduces DistilBERT, a smaller and faster version of the popular BERT language model, which is the basis for the DistilBART model.

1. "BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension" by Mike Lewis, Yinhan Liu, Naman Goyal, Marjan Ghazvininejad, Abdelrahman Mohamed, Omer Levy, Ves Stoyanov, and Luke Zettlemoyer. This paper describes the original BART model, which is the basis for DistilBART, and its use in various natural language processing tasks.

1. "Pre-training Transformers as Energy-Based Denoisers on Natural Language Sequence" by Kevin Clark, Minh-Thang Luong, Quoc V. Le, and Christopher D. Manning. This paper proposes a method for training language models like BERT and BART as denoisers, which is used in the DistilBART model to improve its summarization capabilities.

1. "Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer" by Colin Raffel, Noam Shazeer, Adam Roberts, Katherine Lee, Sharan Narang, Michael Matena, Yanqi Zhou, Wei Li, and Peter J. Liu. This paper introduces the T5 model, a text-to-text transformer that can perform a wide range of natural language processing tasks, including text summarization using techniques similar to those used in DistilBART.

These papers provide a good introduction to the development and capabilities of DistilBART and related language models.



In [1]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.3 MB/s eta 0:00:00


In [2]:
text ="""
 India's Health Ministry has announced that the country's COVID-19 vaccination drive will now be expanded to include people over the age of 60 and those over 45 with co-morbidities. The move is expected to cover an additional 270 million people, making it one of the largest vaccination drives in the world.The decision was taken after a meeting of the National Expert Group on Vaccine Administration for COVID-19 (NEGVAC), which recommended the expansion of the vaccination program. The NEGVAC also suggested that private hospitals may be allowed to administer the vaccine, although the details of this are yet to be finalized.India began its vaccination drive in mid-January, starting with healthcare and frontline workers. Since then, over 13 million doses have been administered across the country. However, the pace of the vaccination drive has been slower than expected, with concerns raised over vaccine hesitancy and logistical challenges.The expansion of the vaccination drive to include the elderly and those with co-morbidities is a major step towards achieving herd immunity and controlling the spread of the virus in India. The Health Ministry has also urged eligible individuals to come forward and get vaccinated at the earliest.India has reported over 11 million cases of COVID-19, making it the second-worst affected country in the world after the United States. The country's daily case count has been declining in recent weeks, but experts have warned that the pandemic is far from over and that precautions need to be maintained.
In summary, India's Health Ministry has announced that the country's COVID-19 vaccination drive will be expanded to include people over 60 and those over 45 with co-morbidities, covering an additional 270 million people. The decision was taken after a meeting of the National Expert Group on Vaccine Administration for COVID-19, and is a major step towards achieving herd immunity and controlling the spread of the virus in India."""

In [3]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [4]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))

In [6]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [15]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))
  result = ''.join(tokenizer.decode(*output, skip_special_tokens=True))

 India's COVID-19 vaccination drive will now be expanded to include people over 60 and those over 45 with co-morbidities. The move is expected to cover an additional 270 million people, making it one of the largest vaccination drives in the world. India began its vaccination drive in mid-January, starting with healthcare and frontline workers. Since then, over 13 million doses have been administered across the country.


In [17]:
result

" India's COVID-19 vaccination drive will now be expanded to include people over 60 and those over 45 with co-morbidities. The move is expected to cover an additional 270 million people, making it one of the largest vaccination drives in the world. India began its vaccination drive in mid-January, starting with healthcare and frontline workers. Since then, over 13 million doses have been administered across the country."

In [10]:
!pip install scikit-learn
!pip install rouge
!pip install nltk
from rouge import Rouge 
import nltk
import nltk.translate.bleu_score as bleu
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
rouge = Rouge()
scores = rouge.get_scores(result, text)
print("ROUGE Score:")
print("Precision: {:.3f}".format(scores[0]['rouge-1']['p']))
print("Recall: {:.3f}".format(scores[0]['rouge-1']['r']))
print("F1-Score: {:.3f}".format(scores[0]['rouge-1']['f']))

ROUGE Score:
Precision: 1.000
Recall: 0.358
F1-Score: 0.527


In [20]:
from nltk.translate.bleu_score import sentence_bleu

def summary_to_sentences(summ):
    # Split the summary into sentences using the '.' character as a separator
    sentences = summ.split('.')
    
    # Convert each sentence into a list of words
    sentence_lists = [sentence.split() for sentence in sentences]
    
    return sentence_lists

def paragraph_to_wordlist(paragraph):
    # Split the paragraph into words using whitespace as a separator
    words = paragraph.split()
    return words

reference_paragraph = text
reference_summary = summary_to_sentences(reference_paragraph)
predicted_paragraph = result
predicted_summary = paragraph_to_wordlist(predicted_paragraph)

score = sentence_bleu(reference_summary, predicted_summary)
print(score)
    

0.8246339537677979


In [21]:
print("BLEU Score: {:.3f}".format(score))

BLEU Score: 0.825
